In [2]:
!wget https://pjreddie.com/media/files/yolov3.weights

--2021-08-14 17:35:22--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights.1’

yolov3.weights.1    100%[===================>] 236.52M  23.3MB/s    in 11s     

2021-08-14 17:35:34 (21.7 MB/s) - ‘yolov3.weights.1’ saved [248007048/248007048]



In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import cv2
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
IMAGE_SIZE = 768
IMAGE_WIDTH = IMAGE_SIZE
IMAGE_HEIGHT = IMAGE_SIZE

YOLO_MAX_BOXES = 100
YOLO_IOU_THRESHOLD = 0.5
YOLO_SCORE_THRESHOLD = 0.5
# define the anchors
#anchors = [[[116,90], [156,198], [373,326]], [[30,61], [62,45], [59,119]], [[10,13], [16,30], [33,23]]]

# define the probability threshold for detected objects
#class_threshold = 0.6

# YoloV3 outputs
#FEATURE_SHAPE = [(24, 24), (48, 48), (96, 96)]

KAGGLE_INPUT = '/kaggle/input/airbus-ship-detection-train-set-70/'
KAGGLE_INPUT_IMAGE_TRAIN = '../input/airbus-ship-detection-train-set-70/train_v3/train_v3/Images/'

N_CLASSES = 1
# 3 слоя, в одной ячейке: x,y,w,h,obj,class
CELL_FEATURE_SIZE = (5 + N_CLASSES) * 3

YOLO_ANCHORS = np.array([(10, 13), (16, 30), (33, 23), (30, 61), (62, 45),
                         (59, 119), (116, 90), (156, 198), (373, 326)],
                        np.float32) / 416
YOLO_ANCHOR_MASKS = np.array([[6, 7, 8], [3, 4, 5], [0, 1, 2]])

In [5]:
def rle2bbox(rle, shape):
    
    a = np.fromiter(rle.split(), dtype=np.uint)
    a = a.reshape((-1, 2))
    a[:,0] -= 1
    
    y0 = a[:,0] % shape[0]
    y1 = y0 + a[:,1]
    if np.any(y1 > shape[0]):
        y0 = 0
        y1 = shape[0]
    else:
        y0 = np.min(y0)
        y1 = np.max(y1)
    
    x0 = a[:,0] // shape[0]
    x1 = (a[:,0] + a[:,1]) // shape[0]
    x0 = np.min(x0)
    x1 = np.max(x1)
    
    if x1 > shape[1]:
        raise ValueError("invalid RLE or image dimensions: x1=%d > shape[1]=%d" % (
            x1, shape[1]
        ))
        
    return [x0/IMAGE_WIDTH, y0/IMAGE_HEIGHT, x1/IMAGE_WIDTH, y1/IMAGE_HEIGHT]

#     xc = (x0+x1)/(2*IMAGE_WIDTH)
#     yc = (y0+y1)/(2*IMAGE_HEIGHT)
#     w = np.abs(x1-x0)/IMAGE_WIDTH
#     h = np.abs(y1-y0)/IMAGE_HEIGHT
#     return [xc, yc, h, w]

In [ ]:
# TODO: переделать под наш старый формат, см. rle2bbox
def load_boxes():
    boxes = dict()
    for path in glob('../input/fish-json/*.json'):
        label = os.path.basename(path).split('_', 1)[0]
        with open(path) as src:
            boxes[label] = json.load(src)
            for annotation in boxes[label]:
                basename = os.path.basename(annotation['filename'])
                annotation['filename'] = os.path.join(TRAIN_PREFIX, label.upper(), basename)
            for annotation in boxes[label]:
                for rect in annotation['annotations']:
                    rect['x'] += rect['width'] / 2
                    rect['y'] += rect['height'] / 2
    return boxes

In [6]:
def aggregate_bb(x):    
    if len(x) == 1 and x.iloc[0] is np.NaN:
        return list()
    
    bb = list(x)
#     for item in x:
# #         item_bb = []
# #         item_bb = {
# #             'x': item[0],
# #             'y': item[1],
# #             'height': item[2],
# #             'width': item[3]
# #         }
#         bb.append(item)
    return bb

In [7]:
ships = pd.read_csv(KAGGLE_INPUT + "train_ship_segmentations_v3.csv")
ships["Boundingbox"] = ships["EncodedPixels"].apply(lambda x:rle2bbox(x,(IMAGE_SIZE,IMAGE_SIZE)) if isinstance(x,str) else np.NaN)

ships_new = ships.groupby(ships['ImageId']).aggregate({'Boundingbox': aggregate_bb, 'ImageId': 'first'})
# ships_without_nans = ships_new[ships_new['Boundingbox'] is not np.NaN]
# ships_without_nans = ships_new[len(ships_new['Boundingbox']) > 0]
# ships_without_nans = ships_new.mask(lambda x: len(x['Boundingbox']) > 0)
ships_without_nans = ships_new[ships_new.apply(lambda x: len(x['Boundingbox']) > 0, axis=1)]
ships_without_nans

,Boundingbox,ImageId
ImageId,,
000155de5.jpg,"[[0.4479166666666667, 0.60546875, 0.58203125, ...",000155de5.jpg
000194a2d.jpg,"[[0.6106770833333334, 0.3736979166666667, 0.63...",000194a2d.jpg
0002756f7.jpg,"[[0.43359375, 0.046875, 0.4557291666666667, 0....",0002756f7.jpg
000532683.jpg,"[[0.77734375, 0.5989583333333334, 0.78515625, ...",000532683.jpg
00053c6ba.jpg,"[[0.9583333333333334, 0.16145833333333334, 0.9...",00053c6ba.jpg
...,...,...
fff77c602.jpg,"[[0.22526041666666666, 0.059895833333333336, 0...",fff77c602.jpg
fff909871.jpg,"[[0.7473958333333334, 0.2747395833333333, 0.79...",fff909871.jpg
fffcb6464.jpg,"[[0.3776041666666667, 0.07161458333333333, 0.4...",fffcb6464.jpg


In [8]:
# based on https://github.com/experiencor/keras-yolo3
import struct
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import ZeroPadding2D
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import add, concatenate
from tensorflow.keras.models import Model

from tensorflow.keras.applications import vgg16

from tensorflow.keras.losses import (
    binary_crossentropy,
    sparse_categorical_crossentropy
)



def _meshgrid(n_a, n_b):

    return [
        tf.reshape(tf.tile(tf.range(n_a), [n_b]), (n_b, n_a)),
        tf.reshape(tf.repeat(tf.range(n_b), n_a), (n_b, n_a))
    ]


def yolo_boxes(pred, anchors, classes):
    # pred: (batch_size, grid, grid, anchors, (x, y, w, h, obj, ...classes))
    grid_size = tf.shape(pred)[1:3]
    box_xy, box_wh, objectness, class_probs = tf.split(
        pred, (2, 2, 1, classes), axis=-1)

    box_xy = tf.sigmoid(box_xy)
    objectness = tf.sigmoid(objectness)
    class_probs = tf.sigmoid(class_probs)
    pred_box = tf.concat((box_xy, box_wh), axis=-1)  # original xywh for loss

    # !!! grid[x][y] == (y, x)
    grid = _meshgrid(grid_size[1],grid_size[0])
    grid = tf.expand_dims(tf.stack(grid, axis=-1), axis=2)  # [gx, gy, 1, 2]

    box_xy = (box_xy + tf.cast(grid, tf.float32)) / \
        tf.cast(grid_size, tf.float32)
    box_wh = tf.exp(box_wh) * anchors

    box_x1y1 = box_xy - box_wh / 2
    box_x2y2 = box_xy + box_wh / 2
    bbox = tf.concat([box_x1y1, box_x2y2], axis=-1)

    return bbox, objectness, class_probs, pred_box


def yolo_nms(outputs, anchors, masks, classes):
    # boxes, conf, type
    b, c, t = [], [], []

    for o in outputs:
        b.append(tf.reshape(o[0], (tf.shape(o[0])[0], -1, tf.shape(o[0])[-1])))
        c.append(tf.reshape(o[1], (tf.shape(o[1])[0], -1, tf.shape(o[1])[-1])))
        t.append(tf.reshape(o[2], (tf.shape(o[2])[0], -1, tf.shape(o[2])[-1])))

    bbox = tf.concat(b, axis=1)
    confidence = tf.concat(c, axis=1)
    class_probs = tf.concat(t, axis=1)

    scores = confidence * class_probs

    dscores = tf.squeeze(scores, axis=0)
    scores = tf.reduce_max(dscores,[1])
    bbox = tf.reshape(bbox,(-1,4))
    classes = tf.argmax(dscores,1)
    selected_indices, selected_scores = tf.image.non_max_suppression_with_scores(
        boxes=bbox,
        scores=scores,
        max_output_size=YOLO_MAX_BOXES,
        iou_threshold=YOLO_IOU_THRESHOLD,
        score_threshold=YOLO_SCORE_THRESHOLD,
        soft_nms_sigma=0.5
    )
    
    num_valid_nms_boxes = tf.shape(selected_indices)[0]

    selected_indices = tf.concat([selected_indices,tf.zeros(YOLO_MAX_BOXES-num_valid_nms_boxes, tf.int32)], 0)
    selected_scores = tf.concat([selected_scores,tf.zeros(YOLO_MAX_BOXES-num_valid_nms_boxes,tf.float32)], -1)

    boxes=tf.gather(bbox, selected_indices)
    boxes = tf.expand_dims(boxes, axis=0)
    scores=selected_scores
    scores = tf.expand_dims(scores, axis=0)
    classes = tf.gather(classes,selected_indices)
    classes = tf.expand_dims(classes, axis=0)
    valid_detections=num_valid_nms_boxes
    valid_detections = tf.expand_dims(valid_detections, axis=0)

    return boxes, scores, classes, valid_detections

 
def _conv_block(inp, convs, skip=True):
    x = inp
    count = 0
    for conv in convs:
        if count == (len(convs) - 2) and skip:
            skip_connection = x
        count += 1
        if conv['stride'] > 1: x = ZeroPadding2D(((1,0),(1,0)))(x) # peculiar padding as darknet prefer left and top
        x = Conv2D(conv['filter'],
                   conv['kernel'],
                   strides=conv['stride'],
                   padding='valid' if conv['stride'] > 1 else 'same', # peculiar padding as darknet prefer left and top
                   name='conv_' + str(conv['layer_idx']),
                   use_bias=False if conv['bnorm'] else True)(x)
        if conv['bnorm']: x = BatchNormalization(epsilon=0.001, name='bnorm_' + str(conv['layer_idx']))(x)
        if conv['leaky']: x = LeakyReLU(alpha=0.1, name='leaky_' + str(conv['layer_idx']))(x)
    return add([skip_connection, x]) if skip else x
 
def make_yolov3_model():
    input_image = Input(shape=(None, None, 3))
    # Layer  0 => 4
    x = _conv_block(input_image, [{'filter': 32, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 0},
                                  {'filter': 64, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 1},
                                  {'filter': 32, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 2},
                                  {'filter': 64, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 3}])
    # Layer  5 => 8
    x = _conv_block(x, [{'filter': 128, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 5},
                        {'filter':  64, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 6},
                        {'filter': 128, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 7}])
    # Layer  9 => 11
    x = _conv_block(x, [{'filter':  64, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 9},
                        {'filter': 128, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 10}])
    # Layer 12 => 15
    x = _conv_block(x, [{'filter': 256, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 12},
                        {'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 13},
                        {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 14}])
    # Layer 16 => 36
    for i in range(7):
        x = _conv_block(x, [{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 16+i*3},
                            {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 17+i*3}])
    skip_36 = x
    # Layer 37 => 40
    x = _conv_block(x, [{'filter': 512, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 37},
                        {'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 38},
                        {'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 39}])
    # Layer 41 => 61
    for i in range(7):
        x = _conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 41+i*3},
                            {'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 42+i*3}])
    skip_61 = x
    # Layer 62 => 65
    x = _conv_block(x, [{'filter': 1024, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 62},
                        {'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 63},
                        {'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 64}])
    # Layer 66 => 74
    for i in range(3):
        x = _conv_block(x, [{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 66+i*3},
                            {'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 67+i*3}])
    # Layer 75 => 79
    x = _conv_block(x, [{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 75},
                        {'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 76},
                        {'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 77},
                        {'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 78},
                        {'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 79}], skip=False)
    # Layer 80 => 82
    yolo_82 = _conv_block(x, [{'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 80},
                              {'filter':  CELL_FEATURE_SIZE, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 81}], skip=False)
    # Layer 83 => 86
    x = _conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 84}], skip=False)
    x = UpSampling2D(2)(x)
    x = concatenate([x, skip_61])
    # Layer 87 => 91
    x = _conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 87},
                        {'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 88},
                        {'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 89},
                        {'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 90},
                        {'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 91}], skip=False)
    # Layer 92 => 94
    yolo_94 = _conv_block(x, [{'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 92},
                              {'filter': CELL_FEATURE_SIZE, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 93}], skip=False)
    # Layer 95 => 98
    x = _conv_block(x, [{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True,   'layer_idx': 96}], skip=False)
    x = UpSampling2D(2)(x)
    x = concatenate([x, skip_36])
    # Layer 99 => 106
    yolo_106 = _conv_block(x, [{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 99},
                               {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 100},
                               {'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 101},
                               {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 102},
                               {'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 103},
                               {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 104},
                               {'filter': CELL_FEATURE_SIZE, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 105}], skip=False)
    
    model = Model(input_image, [yolo_82, yolo_94, yolo_106])
    return model
 
class WeightReader:
    def __init__(self, weight_file):
        with open(weight_file, 'rb') as w_f:
            major,    = struct.unpack('i', w_f.read(4))
            minor,    = struct.unpack('i', w_f.read(4))
            revision, = struct.unpack('i', w_f.read(4))
            if (major*10 + minor) >= 2 and major < 1000 and minor < 1000:
                w_f.read(8)
            else:
                w_f.read(4)
            transpose = (major > 1000) or (minor > 1000)
            binary = w_f.read()
        self.offset = 0
        self.all_weights = np.frombuffer(binary, dtype='float32')
 
    def read_bytes(self, size):
        self.offset = self.offset + size
        return self.all_weights[self.offset-size:self.offset]
 
    def load_weights(self, model):
        for i in range(106):
            try:
                conv_layer = model.get_layer('conv_' + str(i))
                print("loading weights of convolution #" + str(i))
                if i not in [81, 93, 105]:
                    norm_layer = model.get_layer('bnorm_' + str(i))
                    size = np.prod(norm_layer.get_weights()[0].shape)
                    beta  = self.read_bytes(size) # bias
                    gamma = self.read_bytes(size) # scale
                    mean  = self.read_bytes(size) # mean
                    var   = self.read_bytes(size) # variance
                    weights = norm_layer.set_weights([gamma, beta, mean, var])
                if len(conv_layer.get_weights()) > 1:
                    bias   = self.read_bytes(np.prod(conv_layer.get_weights()[1].shape))
                    kernel = self.read_bytes(np.prod(conv_layer.get_weights()[0].shape))
                    if i in [81, 93, 105]:
                        bias = bias[:CELL_FEATURE_SIZE]
                        kernel = kernel[..., :CELL_FEATURE_SIZE]
                        
                    kernel = kernel.reshape(list(reversed(conv_layer.get_weights()[0].shape)))
                    kernel = kernel.transpose([2,3,1,0])
                    conv_layer.set_weights([kernel, bias])
#                     if i in [81, 93, 105]:
#                         print("AAAAAAAA"*5, bias.shape, kernel.shape)
                else:
                    kernel = self.read_bytes(np.prod(conv_layer.get_weights()[0].shape))
                    kernel = kernel.reshape(list(reversed(conv_layer.get_weights()[0].shape)))
                    kernel = kernel.transpose([2,3,1,0])
                    conv_layer.set_weights([kernel])
                    
                if i in [80, 81, 92, 93, 99, 100, 101, 102, 103, 104, 105]:
                    conv_layer.trainable = True
                else:
                    conv_layer.trainable = False
            except ValueError:
                print("no convolution #" + str(i))
 
    def reset(self):
        self.offset = 0
 
# define the model
model = make_yolov3_model()
# load the model weights
weight_reader = WeightReader('yolov3.weights')
# set the model weights into the model
weight_reader.load_weights(model)
# save the model to file
model.save('model.h5')

loading weights of convolution #0
loading weights of convolution #1
loading weights of convolution #2
loading weights of convolution #3
no convolution #4
loading weights of convolution #5
loading weights of convolution #6
loading weights of convolution #7
no convolution #8
loading weights of convolution #9
loading weights of convolution #10
no convolution #11
loading weights of convolution #12
loading weights of convolution #13
loading weights of convolution #14
no convolution #15
loading weights of convolution #16
loading weights of convolution #17
no convolution #18
loading weights of convolution #19
loading weights of convolution #20
no convolution #21
loading weights of convolution #22
loading weights of convolution #23
no convolution #24
loading weights of convolution #25
loading weights of convolution #26
no convolution #27
loading weights of convolution #28
loading weights of convolution #29
no convolution #30
loading weights of convolution #31
loading weights of convolution #32

In [9]:
def load_img(file_name, target_size=(IMAGE_WIDTH, IMAGE_HEIGHT)):
#     file_name = sub_df.iloc[0].loc['ImageId']
    img = cv2.imread(KAGGLE_INPUT_IMAGE_TRAIN + file_name, cv2.IMREAD_COLOR)[...,::-1]
#     img = cv2.imread(path, cv2.IMREAD_COLOR)[...,::-1]
    img_shape = img.shape
    img_resized = cv2.resize(img, target_size)
    return img_shape, img_resized.astype(np.float32) / 255.0 #vgg16.preprocess_input(img_resized.astype(np.float32))


In [10]:


# def parse_tfrecord(tfrecord, class_table, size):
#     x = tf.io.parse_single_example(tfrecord, IMAGE_FEATURE_MAP)
#     x_train = tf.image.decode_jpeg(x['image/encoded'], channels=3)
#     x_train = tf.image.resize(x_train, (size, size))

#     class_text = tf.sparse.to_dense(
#         x['image/object/class/text'], default_value='')
#     labels = tf.cast(class_table.lookup(class_text), tf.float32)
#     y_train = tf.stack([tf.sparse.to_dense(x['image/object/bbox/xmin']),
#                         tf.sparse.to_dense(x['image/object/bbox/ymin']),
#                         tf.sparse.to_dense(x['image/object/bbox/xmax']),
#                         tf.sparse.to_dense(x['image/object/bbox/ymax']),
#                         labels], axis=1)

#     paddings = [[0, YOLO_MAX_BOXES - tf.shape(y_train)[0]], [0, 0]]
#     y_train = tf.pad(y_train, paddings)

#     return x_train, y_train


# def load_tfrecord_dataset(file_pattern, class_file, size=416):
#     LINE_NUMBER = -1  # TODO: use tf.lookup.TextFileIndex.LINE_NUMBER
#     class_table = tf.lookup.StaticHashTable(tf.lookup.TextFileInitializer(
#         class_file, tf.string, 0, tf.int64, LINE_NUMBER, delimiter="\n"), -1)

#     files = tf.data.Dataset.list_files(file_pattern)
#     dataset = files.flat_map(tf.data.TFRecordDataset)
#     return dataset.map(lambda x: parse_tfrecord(x, class_table, size))

In [11]:
# import tensorflow as tf

# @tf.function
# def transform_targets_for_output(y_true, grid_size, anchor_idxs):
#     # y_true: (N, boxes, (x1, y1, x2, y2, class, best_anchor))
#     N = tf.shape(y_true)[0]

#     # y_true_out: (N, grid, grid, anchors, [x1, y1, x2, y2, obj, class])
#     y_true_out = tf.zeros(
#         (N, grid_size, grid_size, tf.shape(anchor_idxs)[0], 6))

#     anchor_idxs = tf.cast(anchor_idxs, tf.int32)

#     indexes = tf.TensorArray(tf.int32, 1, dynamic_size=True)
#     updates = tf.TensorArray(tf.float32, 1, dynamic_size=True)
#     idx = 0
#     for i in tf.range(N):
#         for j in tf.range(tf.shape(y_true)[1]):
#             if tf.equal(y_true[i][j][2], 0):
#                 continue
#             anchor_eq = tf.equal(
#                 anchor_idxs, tf.cast(y_true[i][j][5], tf.int32))

#             if tf.reduce_any(anchor_eq):
#                 box = y_true[i][j][0:4]
#                 box_xy = (y_true[i][j][0:2] + y_true[i][j][2:4]) / 2

#                 anchor_idx = tf.cast(tf.where(anchor_eq), tf.int32)
#                 grid_xy = tf.cast(box_xy // (1/grid_size), tf.int32)

#                 # grid[y][x][anchor] = (tx, ty, bw, bh, obj, class)
#                 indexes = indexes.write(
#                     idx, [i, grid_xy[1], grid_xy[0], anchor_idx[0][0]])
#                 updates = updates.write(
#                     idx, [box[0], box[1], box[2], box[3], 1, y_true[i][j][4]])
#                 idx += 1

#     # tf.print(indexes.stack())
#     # tf.print(updates.stack())

#     return tf.tensor_scatter_nd_update(
#         y_true_out, indexes.stack(), updates.stack())


# def transform_targets(y_train, anchors, anchor_masks, size):
#     y_outs = []
#     grid_size = size // 32

#     # calculate anchor index for true boxes
#     anchors = tf.cast(anchors, tf.float32)
#     anchor_area = anchors[..., 0] * anchors[..., 1]
#     box_wh = y_train[..., 2:4] - y_train[..., 0:2]
#     box_wh = tf.tile(tf.expand_dims(box_wh, -2),
#                      (1, 1, tf.shape(anchors)[0], 1))
#     box_area = box_wh[..., 0] * box_wh[..., 1]
#     intersection = tf.minimum(box_wh[..., 0], anchors[..., 0]) * \
#         tf.minimum(box_wh[..., 1], anchors[..., 1])
#     iou = intersection / (box_area + anchor_area - intersection)
#     anchor_idx = tf.cast(tf.argmax(iou, axis=-1), tf.float32)
#     anchor_idx = tf.expand_dims(anchor_idx, axis=-1)

#     y_train = tf.concat([y_train, anchor_idx], axis=-1)

#     for anchor_idxs in anchor_masks:
#         y_outs.append(transform_targets_for_output(
#             y_train, grid_size, anchor_idxs))
#         grid_size *= 2

#     return tuple(y_outs)


# def transform_images(x_train, size):
#     x_train = tf.image.resize(x_train, (size, size))
#     x_train = x_train / 255
#     return x_train


In [12]:

def broadcast_iou(box_1, box_2):
    # box_1: (..., (x1, y1, x2, y2))
    # box_2: (N, (x1, y1, x2, y2))

    # broadcast boxes
    box_1 = tf.expand_dims(box_1, -2)
    box_2 = tf.expand_dims(box_2, 0)
    # new_shape: (..., N, (x1, y1, x2, y2))
    new_shape = tf.broadcast_dynamic_shape(tf.shape(box_1), tf.shape(box_2))
    box_1 = tf.broadcast_to(box_1, new_shape)
    box_2 = tf.broadcast_to(box_2, new_shape)

    int_w = tf.maximum(tf.minimum(box_1[..., 2], box_2[..., 2]) -
                       tf.maximum(box_1[..., 0], box_2[..., 0]), 0)
    int_h = tf.maximum(tf.minimum(box_1[..., 3], box_2[..., 3]) -
                       tf.maximum(box_1[..., 1], box_2[..., 1]), 0)
    int_area = int_w * int_h
    box_1_area = (box_1[..., 2] - box_1[..., 0]) * \
        (box_1[..., 3] - box_1[..., 1])
    box_2_area = (box_2[..., 2] - box_2[..., 0]) * \
        (box_2[..., 3] - box_2[..., 1])
    return int_area / (box_1_area + box_2_area - int_area)

def yolo_boxes(pred, anchors, classes):
    # pred: (batch_size, grid, grid, anchors, (x, y, w, h, obj, ...classes))
#     print("CHECK")
    grid_size = tf.shape(pred)[1:3]
    box_xy, box_wh, objectness, class_probs = tf.split(
        pred, (2, 2, 1, classes), axis=-1)
#     print("pred", pred)
#     print("obj", objectness)

    box_xy = tf.sigmoid(box_xy)
    objectness = tf.sigmoid(objectness)
    class_probs = tf.sigmoid(class_probs)
    pred_box = tf.concat((box_xy, box_wh), axis=-1)  # original xywh for loss

    # !!! grid[x][y] == (y, x)
    grid = _meshgrid(grid_size[1],grid_size[0])
    grid = tf.expand_dims(tf.stack(grid, axis=-1), axis=2)  # [gx, gy, 1, 2]

    box_xy = (box_xy + tf.cast(grid, tf.float32)) / \
        tf.cast(grid_size, tf.float32)
    box_wh = tf.exp(box_wh) * anchors

    box_x1y1 = box_xy - box_wh / 2
    box_x2y2 = box_xy + box_wh / 2
    bbox = tf.concat([box_x1y1, box_x2y2], axis=-1)

    return bbox, objectness, class_probs, pred_box


def YoloLoss(anchors, classes=80, ignore_thresh=0.5):
#     print('CHECK yoloLOSS')
    def yolo_loss(y_true, y_pred):
        # 1. transform all pred outputs
        # y_pred: (batch_size, grid, grid, anchors, (x, y, w, h, obj, ...cls))
#         print('y_true', y_true)
#         print(y_true)
#         print('y_pred, before', y_pred)
        y_pred = tf.reshape(y_pred, (-1, -1, -1, 3, 5 + N_CLASSES))
        y_true = tf.reshape(y_true, (-1, -1, -1, 3, 5 + N_CLASSES))
#         print('y_pred, after', y_pred)
        pred_box, pred_obj, pred_class, pred_xywh = yolo_boxes(
            y_pred, anchors, classes)
        pred_xy = pred_xywh[..., 0:2]
        pred_wh = pred_xywh[..., 2:4]

        # 2. transform all true outputs
        # y_true: (batch_size, grid, grid, anchors, (x1, y1, x2, y2, obj, cls))
        true_box, true_obj, true_class_idx = tf.split(
            y_true, (4, 1, 1), axis=-1)
        true_xy = (true_box[..., 0:2] + true_box[..., 2:4]) / 2
        true_wh = true_box[..., 2:4] - true_box[..., 0:2]

        # give higher weights to small boxes
        box_loss_scale = 2 - true_wh[..., 0] * true_wh[..., 1]

        # 3. inverting the pred box equations
        grid_size = tf.shape(y_true)[1]
        grid = tf.meshgrid(tf.range(grid_size), tf.range(grid_size))
        grid = tf.expand_dims(tf.stack(grid, axis=-1), axis=2)
        true_xy = true_xy * tf.cast(grid_size, tf.float32) - \
            tf.cast(grid, tf.float32)
        true_wh = tf.math.log(true_wh / anchors)
        true_wh = tf.where(tf.math.is_inf(true_wh),
                           tf.zeros_like(true_wh), true_wh)

        # 4. calculate all masks
        obj_mask = tf.squeeze(true_obj, -1)
        # ignore false positive when iou is over threshold
        best_iou = tf.map_fn(
            lambda x: tf.reduce_max(broadcast_iou(x[0], tf.boolean_mask(
                x[1], tf.cast(x[2], tf.bool))), axis=-1),
            (pred_box, true_box, obj_mask),
            tf.float32)
        ignore_mask = tf.cast(best_iou < ignore_thresh, tf.float32)

        # 5. calculate all losses
        xy_loss = obj_mask * box_loss_scale * \
            tf.reduce_sum(tf.square(true_xy - pred_xy), axis=-1)
        wh_loss = obj_mask * box_loss_scale * \
            tf.reduce_sum(tf.square(true_wh - pred_wh), axis=-1)
#         print('true_obj', true_obj)
#         print('pred_obj', pred_obj)
        obj_loss = binary_crossentropy(true_obj, pred_obj)
        obj_loss = obj_mask * obj_loss + \
            (1 - obj_mask) * ignore_mask * obj_loss
        # TODO: use binary_crossentropy instead
        class_loss = obj_mask * sparse_categorical_crossentropy(
            true_class_idx, pred_class)

        # 6. sum over (batch, gridx, gridy, anchors) => (batch, 1)
        xy_loss = tf.reduce_sum(xy_loss, axis=(1, 2, 3))
        wh_loss = tf.reduce_sum(wh_loss, axis=(1, 2, 3))
        obj_loss = tf.reduce_sum(obj_loss, axis=(1, 2, 3))
        class_loss = tf.reduce_sum(class_loss, axis=(1, 2, 3))

        return xy_loss + wh_loss + obj_loss + class_loss
    return yolo_loss

In [13]:
n = np.array([[1, 2, 3, 4], [2, 3, 5, 6]])
print(n.shape)
n = n.reshape((-1, 2, 2))
print(n.shape)


(2, 4)
(2, 2, 2)


In [14]:
@tf.function
def transform_targets_for_output(y_true, grid_size, anchor_idxs):
    # y_true: (N, boxes, (x1, y1, x2, y2, class, best_anchor))
    N = tf.shape(y_true)[0]

    # y_true_out: (N, grid, grid, anchors, [x1, y1, x2, y2, obj, class])
    y_true_out = tf.zeros(
        (N, grid_size, grid_size, tf.shape(anchor_idxs)[0], 6))

    anchor_idxs = tf.cast(anchor_idxs, tf.int32)

    indexes = tf.TensorArray(tf.int32, 1, dynamic_size=True)
    updates = tf.TensorArray(tf.float32, 1, dynamic_size=True)
    idx = 0
    for i in tf.range(N):
        for j in tf.range(tf.shape(y_true)[1]):
            if tf.equal(y_true[i][j][2], 0):
                continue
            anchor_eq = tf.equal(
                anchor_idxs, tf.cast(y_true[i][j][5], tf.int32))

            if tf.reduce_any(anchor_eq):
                box = y_true[i][j][0:4]
                box_xy = (y_true[i][j][0:2] + y_true[i][j][2:4]) / 2

                anchor_idx = tf.cast(tf.where(anchor_eq), tf.int32)
                grid_xy = tf.cast(box_xy // (1/grid_size), tf.int32)

                # grid[y][x][anchor] = (tx, ty, bw, bh, obj, class)
                indexes = indexes.write(
                    idx, [i, grid_xy[1], grid_xy[0], anchor_idx[0][0]])
                updates = updates.write(
                    idx, [box[0], box[1], box[2], box[3], 1, y_true[i][j][4]])
                idx += 1

    # tf.print(indexes.stack())
    # tf.print(updates.stack())
    _y = tf.tensor_scatter_nd_update(
        y_true_out, indexes.stack(), updates.stack())
    return tf.reshape(_y, (N, grid_size, grid_size, tf.shape(anchor_idxs)[0] * 6))
#     return tf.tensor_scatter_nd_update(
#         y_true_out, indexes.stack(), updates.stack())


def transform_targets(y_train, anchors, anchor_masks, size):
    y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
#     y_train = np.array(y_train)
    y_outs = []
    grid_size = size // 32

    # calculate anchor index for true boxes
    anchors = tf.cast(anchors, tf.float32)
    anchor_area = anchors[..., 0] * anchors[..., 1]
    box_wh = y_train[..., 2:4] - y_train[..., 0:2]
    box_wh = tf.tile(tf.expand_dims(box_wh, -2),
                     (1, 1, tf.shape(anchors)[0], 1))
    box_area = box_wh[..., 0] * box_wh[..., 1]
    intersection = tf.minimum(box_wh[..., 0], anchors[..., 0]) * \
        tf.minimum(box_wh[..., 1], anchors[..., 1])
#     print(box_wh)
#     print(anchors)
    iou = intersection / (box_area + anchor_area - intersection)
    anchor_idx = tf.cast(tf.argmax(iou, axis=-1), tf.float32)
    anchor_idx = tf.expand_dims(anchor_idx, axis=-1)

    y_train = tf.concat([y_train, anchor_idx], axis=-1)

    for anchor_idxs in anchor_masks:
        y_outs.append(transform_targets_for_output(
            y_train, grid_size, anchor_idxs))
        grid_size *= 2

    return tuple(y_outs)

In [15]:
from random import shuffle

def parse_boxes(img_infos):
    N = len (img_infos)
    # TODO!!! Start from here
    # y: (N, boxes, (x1, y1, x2, y2, class))
    # tf.tensor(N, YOLO_MAX_BOXES, 5)
    # for rect in input_data['Boundingbox']:
    #    if rect is not np.NaN:
    #        x0, y0, x1, y1 = rect
    
def transform_to_list(row):
    data = []
    for i in row:
        l = i[:]
        l.append(0.0)
        data.append(l)
        
    if len(row) < YOLO_MAX_BOXES:
        for i in range(YOLO_MAX_BOXES - len(row)):
            data += [[0.0, 0.0, 0.0, 0.0, 0.0]]
            
    return data

def data_generator(train_df, batch_size=32):
    print("START WORK")
    while True:
        X, y = [], []
        for ind, i in train_df.sample(n=batch_size).iterrows():
            img_shape, img = load_img(i['ImageId'])
            data = transform_to_list(i['Boundingbox'])
            
#             if len(data) < YOLO_MAX_BOXES:
#                 data += [[0, 0, 0, 0, 0] * YOLO_MAX_BOXES - 
            
            y.append(data)
#             y.append(transform_targets(data, YOLO_ANCHORS, YOLO_ANCHOR_MASKS, IMAGE_SIZE))
            X.append(img)
    
            
        print("HEREE")
        y = transform_targets(y, YOLO_ANCHORS, YOLO_ANCHOR_MASKS, IMAGE_SIZE)
        
#         print("ISCHEM tshape")
#         for t in y:
#             print(t.shape)
#         print(y.shape, type(y))
        yield np.array(X), y

In [16]:
adam = tf.keras.optimizers.Adam(lr=3e-4, decay=1e-6)

loss = [YoloLoss(YOLO_ANCHORS[mask], classes=1)
        for mask in YOLO_ANCHOR_MASKS]

model.compile(optimizer=adam, 
              loss=loss)
# YoloLoss(tf.convert_to_tensor(YOLO_ANCHORS, dtype=tf.float32), classes=1))

In [ ]:
batch_size = 32

gen = data_generator(ships_without_nans, batch_size=batch_size)
steps_per_epoch = len(ships_without_nans['ImageId']) / batch_size

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'hz_detector.hdf5',
    monitor='loss',
    verbose=1,  
    save_best_only=True, 
    save_weights_only=False,
    mode='auto',
    save_freq=10
)

model.fit(gen, 
          steps_per_epoch=steps_per_epoch,
          epochs=1)

START WORK
HEREE
HEREE
